In [ ]:
import cv2
import os
import shutil
import numpy as np
import zipfile
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tqdm import tqdm
from PIL import Image, ImageFilter, ImageEnhance
from keras.optimizers import Adam
from keras.regularizers import l2

In [ ]:
# Unzipping the image dataset
zip_ref = zipfile.ZipFile('original.zip', 'r')
zip_ref.extractall('dataset')  # Extracting to the 'dataset' folder
zip_ref.close()

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255
)

rotated_dir = 'dataset/scaled'
if not os.path.exists(rotated_dir):
    os.makedirs(rotated_dir)

original_dir = 'dataset/original'
total_images = 20

for i in range(1, total_images + 1):
    img_path = f'{original_dir}/{i:02d}.jpg'
    img = load_img(img_path)
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)

    j = 0
    for batch in datagen.flow(x, batch_size=1, save_to_dir=rotated_dir, save_prefix='scaled', save_format='jpeg'):
        j += 1
        if j >= 10:
            break

In [ ]:
# Define data paths
original_dir = 'dataset/original'
scaled_dir = 'dataset/scaled'
train_dir = 'dataset/train'
validation_dir = 'dataset/validation'

# Create directories for training and validation
for category in ['original', 'scaled']:
    os.makedirs(os.path.join(train_dir, category), exist_ok=True)
    os.makedirs(os.path.join(validation_dir, category), exist_ok=True)

# Function to split data and move files
def split_data(source_folder, train_folder, validation_folder, validation_split=0.2):
    files = os.listdir(source_folder)
    train_files, validation_files = train_test_split(files, test_size=validation_split)

    # Copy files to training directory
    for file in train_files:
        shutil.copy(os.path.join(source_folder, file), os.path.join(train_folder, file))

    # Copy files to validation directory
    for file in validation_files:
        shutil.copy(os.path.join(source_folder, file), os.path.join(validation_folder, file))

# Apply function to original and distorted datasets
split_data(original_dir, os.path.join(train_dir, 'original'), os.path.join(validation_dir, 'original'))
split_data(scaled_dir, os.path.join(train_dir, 'scaled'), os.path.join(validation_dir, 'scaled'))

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=(224, 224),
    batch_size=5,
    class_mode='categorical'
)

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_generator = validation_datagen.flow_from_directory(
    'dataset/validation',
    target_size=(224, 224),
    batch_size=5,
    class_mode='categorical'
)

Found 175 images belonging to 2 classes.
Found 44 images belonging to 2 classes.


In [ ]:
import shutil

def remove_unwanted_dirs(directory):
    checkpoint_dir = os.path.join(directory, '.ipynb_checkpoints')
    if os.path.exists(checkpoint_dir):
        shutil.rmtree(checkpoint_dir)
        print(f"Removed {checkpoint_dir}")

remove_unwanted_dirs(train_dir)
remove_unwanted_dirs(validation_dir)

print("Updated training directories:", os.listdir(train_dir))
print("Updated validation directories:", os.listdir(validation_dir))

Updated training directories: ['original', 'scaled']
Updated validation directories: ['original', 'scaled']


In [ ]:
dataset_dir = 'dataset'
print("Directories in dataset:", os.listdir(dataset_dir))

for category in os.listdir(dataset_dir):
    category_path = os.path.join(dataset_dir, category)
    if os.path.isdir(category_path):
        print(f"Contents of {category}:", os.listdir(category_path))

Directories in dataset: ['original', 'validation', 'train', 'scaled']
Contents of original: ['20.jpg', '13.jpg', '06.jpg', '01.jpg', '03.jpg', '18.jpg', '04.jpg', '15.jpg', '16.jpg', '07.jpg', '11.jpg', '09.jpg', '14.jpg', '02.jpg', '19.jpg', '10.jpg', '12.jpg', '08.jpg', '05.jpg', '17.jpg']
Contents of validation: ['original', 'scaled']
Contents of train: ['original', 'scaled']
Contents of scaled: ['scaled_0_3751.jpeg', 'scaled_0_7650.jpeg', 'scaled_0_35.jpeg', 'scaled_0_8259.jpeg', 'scaled_0_1484.jpeg', 'scaled_0_1064.jpeg', 'scaled_0_9523.jpeg', 'scaled_0_2818.jpeg', 'scaled_0_6747.jpeg', 'scaled_0_6406.jpeg', 'scaled_0_8774.jpeg', 'scaled_0_6323.jpeg', 'scaled_0_4344.jpeg', 'scaled_0_3479.jpeg', 'scaled_0_8052.jpeg', 'scaled_0_8489.jpeg', 'scaled_0_9577.jpeg', 'scaled_0_5768.jpeg', 'scaled_0_4973.jpeg', 'scaled_0_3732.jpeg', 'scaled_0_3962.jpeg', 'scaled_0_6251.jpeg', 'scaled_0_5785.jpeg', 'scaled_0_916.jpeg', 'scaled_0_7011.jpeg', 'scaled_0_3444.jpeg', 'scaled_0_3517.jpeg', 'scale

In [ ]:
import shutil
import os

directories_to_remove = ['original', 'scaled']
base_dir = 'dataset'

for dir_name in directories_to_remove:
    dir_path = os.path.join(base_dir, dir_name)
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
        print(f"Removed directory: {dir_path}")
    else:
        print(f"Directory does not exist, no need to remove: {dir_path}")

print("Updated contents of the dataset directory:", os.listdir(base_dir))

Directory does not exist, no need to remove: dataset/original
Removed directory: dataset/scaled
Updated contents of the dataset directory: ['validation', 'train']


In [ ]:
!pip install tensorflow_addons
import tensorflow_addons as tfa

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = Flatten()(base_model.output)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)  # L2
x = Dropout(0.5)(x)  # Dropout
predictions = Dense(1, activation='sigmoid')(x)

optimizer = tfa.optimizers.AdamW(learning_rate=0.0001, weight_decay=0.001)

# Compile the model
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Prepare training data
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2  # Use 20% of the data as validation set
)

train_generator = train_datagen.flow_from_directory(
    'dataset',
    target_size=(224, 224),
    batch_size=5,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'dataset',
    target_size=(224, 224),
    batch_size=5,
    class_mode='binary',
    subset='validation'
)

# Use early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 6.4 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


58889256/58889256 [==============================] - 0s 0us/step
Found 176 images belonging to 2 classes.
Found 43 images belonging to 2 classes.


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=15,
    epochs=10,
    callbacks=[early_stopping],
    validation_data=validation_generator,
    validation_steps=5
)

# Evaluate the model
results = model.evaluate(validation_generator)
print("Test loss, Test accuracy:", results)

Epoch 1/10
15/15 [==============================] - 77s 5s/step - loss: 5.8019 - accuracy: 0.7606 - val_loss: 9.9698 - val_accuracy: 0.6800
Epoch 2/10
15/15 [==============================] - 62s 4s/step - loss: 11.7996 - accuracy: 0.6533 - val_loss: 7.5555 - val_accuracy: 0.6000
Epoch 3/10
15/15 [==============================] - 65s 4s/step - loss: 10.5991 - accuracy: 0.6267 - val_loss: 7.1520 - val_accuracy: 0.7600
Epoch 4/10
15/15 [==============================] - 65s 4s/step - loss: 9.7862 - accuracy: 0.7467 - val_loss: 3.6466 - val_accuracy: 0.8000
Epoch 5/10
15/15 [==============================] - 57s 4s/step - loss: 8.7549 - accuracy: 0.6338 - val_loss: 4.4411 - val_accuracy: 0.7200
Epoch 6/10
15/15 [==============================] - 61s 4s/step - loss: 6.9955 - accuracy: 0.7200 - val_loss: 2.6625 - val_accuracy: 0.8000
Epoch 7/10
15/15 [==============================] - 65s 4s/step - loss: 6.9775 - accuracy: 0.7333 - val_loss: 5.2025 - val_accuracy: 0.8000
Epoch 8/10
15/15 [